# Introduction
<font color = green> <strong>MUST READ:</strong></font> This Notebook tutorial will allow you to register a TNG manifest file into aws-iot account to enable device connectivity. To run this Notebook successfuly, its required to have AWS CLI loaded with account credentials, Manifest file and Manifest file Signer are generated, and Manifest file is Signed by Manifest signer. The Manifest file related steps will be taken care by TNGTLS Manifest File Generation Notebook.

Before running this Notebook,
1. <font><strong>docs/AWS Test account credentials.csv</strong></font> file should have updated with User credentials to be used.
2. Run TNGTLS Manifest File Generation Notebook to ensure Manifest file generated and signed

### Prerequisites:
This step of the tutorial will attempt to load all the necessary modules and their dependencies on your machine. If the modules are already installed you can safely step over the next Tutorial step.

<font color = orange> <strong>Note</strong></font> 
1. Installation time for prerequisites depends upon system and network speed.
2. Installing prerequisites for the first time takes more time and watch the kernel status for progress. Following image helps to locate the Kernel status,
<center><img src="../../../../assets/notebook/img/kerner_status.png" alt="**Check Kernel Status**" /></center>


3. Installing prerequisites gives the following error and it can be safely ignored. Functionality remains unaffected.
    - <font color = orange> azure-cli 2.0.76 has requirement colorama~=0.4.1, but you'll have colorama 0.3.9 which is incompatible.</font>
    - <font color = orange> azure-cli 2.0.76 has requirement pytz==2019.1, but you'll have pytz 2019.3 which is incompatible. </font>

In [ ]:
import sys, os

home_path = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.realpath(os.getcwd())))))
module_path = os.path.join(home_path, 'assets', 'python')
if not module_path in sys.path:
    sys.path.append(module_path)

from requirements_helper import requirements_installer
obj = requirements_installer(os.path.join(home_path, 'assets', 'requirements.txt'))

### Load the necessary modules and helper functions
This step loads the required modules and helper functions to perform the required operations.

In [ ]:
import json
from helper_aws import *
from Microchip_manifest_handler import *
from gui_aws import *
from ipywidgets import *

The following steps assist to perform
1. Updating AWS account credentials
2. Manifest file upload to AWS account and verify
3. Triggering the AWS GUI to control connected device

#### Updating AWS account credentials (1. Config AWS-CLI)
This button helps to update user AWS credentials to AWS-CLI. These will be used for any interactions with AWS cloud from the PC. This is an option button to execute. This can be skipped if configuration displayed is correct for the User account.

In [ ]:
def get_user_region(b):
    config_aws_cli.button_style = configure_aws_cli(text.value)
    print('--------------------------------------')
def execute_aws_configure(b):
    display(text)
    text.on_submit(get_user_region)
    
config_aws_cli = widgets.Button(description = "Step1. Config AWS-CLI", layout=widgets.Layout(width='auto'))
text = widgets.Text(value='', description='AWS User Region:', style={'description_width': '150px'})
manifest_file = FileUpload(description='Step2 a. Load Manifest JSON File', accept='.json', layout=widgets.Layout(width='auto'), multiple=False)
validation_cert = FileUpload(description='Step2 b. Load Validation CERT File', accept='.crt', layout=widgets.Layout(width='auto'), multiple=False)
manifest_upload = widgets.Button(description = "Step2 c. Upload Manifest File", layout=widgets.Layout(width='auto'))
aws_gui_b = widgets.Button(description = "Step3. AWS GUI")


#### File navigation for Manifest file and certificate (2. Upload Manifest file)
This button helps to load the Manifest file to AWS account. To be able to upload successfully, its required to provide devices Manifest file and Manifest file Signer certificate. On uploading, this step also verifies the upload process by fetching the device certificate(s) from AWS account and validating with Signer certificate.

In [ ]:
def execute_manifest_upload(b):
    check_and_install_policy('Default')
    if not manifest_file.data or not validation_cert.data:
        print('Step2a & Step2b should be executed first before proceeding to this step\r\n')
        return None
    manifest_data = json.loads(manifest_file.data[0])
    cert_data = validation_cert.data[0]                           
    invoke_import_manifest('Default', manifest_data, cert_data)
    invoke_validate_manifest_import(manifest_data, cert_data)
    manifest_upload.button_style = 'success'
    print('--------------------------------------')

#### Trigger AWS GUI (3. AWS GUI)
Before running this step, target hardware should be compiled with the above host name and wifi credentials. Once, the target hardware is programmed and connected to cloud, click this button to pops up a GUI. This GUI can be used to control the LED on the hardware.

In [ ]:
def execute_aws_gui(b):
    thing = widgets.Text(value='', description='Enter device uniqueId:', style={'description_width': '300px'}, layout=Layout(width='50%'))
    thing.on_submit(execute_gui)
    display(thing)
    
def execute_gui(b):
    if(len(b.value) != 18 ):
        print("Entered uniqueID length is not correct and it should be 18 characters")
        return None
    aws_interact_gui('default', b.value)

In [ ]:
print('--------------------------------------')
list_current_configuration()

config_aws_cli.on_click(execute_aws_configure)
manifest_upload.on_click(execute_manifest_upload)
aws_gui_b.on_click(execute_aws_gui)
print('--------------------------------------')

v = widgets.VBox((manifest_file, validation_cert, manifest_upload))
display(widgets.HBox((config_aws_cli, v)))
print('--------------------------------------')
print('Before clicking AWS GUI its required to have Manifest file uploaded and \
Connecting embedded project to cloud by using host name and wifi credentials. \
Click below AWS GUI button ONLY after establishing connection with Cloud')
display(aws_gui_b) 
print('--------------------------------------')